<a href="https://colab.research.google.com/github/CurtCalledBurt/DS-Unit-2-Regression-Classification/blob/master/module4/assignment_regression_classification_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, sys
in_colab = 'google.colab' in sys.modules

# If you're in Colab...
if in_colab:
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification.git
    !git pull origin master
    
    # Install required python packages
    !pip install -r requirements.txt
    
    # Change into directory for module
    os.chdir('module4')

Initialized empty Git repository in /content/.git/
remote: Enumerating objects: 156, done.
remote: Total 156 (delta 0), reused 0 (delta 0), pack-reused 156
Receiving objects: 100% (156/156), 19.30 MiB | 14.23 MiB/s, done.
Resolving deltas: 100% (71/71), done.
From https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification
 * branch            master     -> FETCH_HEAD
 * [new branch]      master     -> origin/master
     |████████████████████████████████| 92kB 7.8MB/s 
     |████████████████████████████████| 112kB 33.3MB/s 
     |████████████████████████████████| 133kB 47.9MB/s 
     |████████████████████████████████| 57.7MB 1.3MB/s 
     |████████████████████████████████| 235kB 44.8MB/s 
     |████████████████████████████████| 614kB 44.3MB/s 
     |████████████████████████████████| 235kB 28.6MB/s 
     |████████████████████████████████| 307kB 54.5MB/s 
     |████████████████████████████████| 51kB 22.9MB/s 
     |████████████████████████████████| 215kB 47.2MB/s 
     |█████████

In [0]:
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
# Read the Tanzania Waterpumps data
# train_features.csv : the training set features
# train_labels.csv : the training set labels
# test_features.csv : the test set features
# sample_submission.csv : a sample submission file in the correct format
    
import pandas as pd

train_features = pd.read_csv('../data/waterpumps/train_features.csv')
train_labels = pd.read_csv('../data/waterpumps/train_labels.csv')
test_features = pd.read_csv('../data/waterpumps/test_features.csv')
sample_submission = pd.read_csv('../data/waterpumps/sample_submission.csv')

assert train_features.shape == (59400, 40)
assert train_labels.shape == (59400, 2)
assert test_features.shape == (14358, 40)
assert sample_submission.shape == (14358, 2)

Lambda School Data Science, Unit 2: Predictive Modeling

# Regression & Classification, Module 4


## Assignment

- [ ] Watch Aaron's [video #1](https://www.youtube.com/watch?v=pREaWFli-5I) (12 minutes) & [video #2](https://www.youtube.com/watch?v=bDQgVt4hFgY) (9 minutes) to learn about the mathematics of Logistic Regression.
- [ ] [Sign up for a Kaggle account](https://www.kaggle.com/), if you don’t already have one. Go to our Kaggle InClass competition website. You will be given the URL in Slack. Go to the Rules page. Accept the rules of the competition.
- [ ] Do train/validate/test split with the Tanzania Waterpumps data.
- [ ] Begin with baselines for classification.
- [ ] Use scikit-learn for logistic regression.
- [ ] Get your validation accuracy score.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.

---


## Stretch Goals

- [ ] Add your own stretch goal(s) !
- [ ] Clean the data. For ideas, refer to [The Quartz guide to bad data](https://github.com/Quartz/bad-data-guide),  a "reference to problems seen in real-world data along with suggestions on how to resolve them." One of the issues is ["Zeros replace missing values."](https://github.com/Quartz/bad-data-guide#zeros-replace-missing-values)
- [ ] Make exploratory visualizations.
- [ ] Do one-hot encoding. For example, you could try `quantity`, `basin`, `extraction_type_class`, and more. (But remember it may not work with high cardinality categoricals.)
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Get and plot your coefficients.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

---

## Data Dictionary 

### Features

Your goal is to predict the operating condition of a waterpoint for each record in the dataset. You are provided the following set of information about the waterpoints:

- `amount_tsh` : Total static head (amount water available to waterpoint)
- `date_recorded` : The date the row was entered
- `funder` : Who funded the well
- `gps_height` : Altitude of the well
- `installer` : Organization that installed the well
- `longitude` : GPS coordinate
- `latitude` : GPS coordinate
- `wpt_name` : Name of the waterpoint if there is one
- `num_private` :  
- `basin` : Geographic water basin
- `subvillage` : Geographic location
- `region` : Geographic location
- `region_code` : Geographic location (coded)
- `district_code` : Geographic location (coded)
- `lga` : Geographic location
- `ward` : Geographic location
- `population` : Population around the well
- `public_meeting` : True/False
- `recorded_by` : Group entering this row of data
- `scheme_management` : Who operates the waterpoint
- `scheme_name` : Who operates the waterpoint
- `permit` : If the waterpoint is permitted
- `construction_year` : Year the waterpoint was constructed
- `extraction_type` : The kind of extraction the waterpoint uses
- `extraction_type_group` : The kind of extraction the waterpoint uses
- `extraction_type_class` : The kind of extraction the waterpoint uses
- `management` : How the waterpoint is managed
- `management_group` : How the waterpoint is managed
- `payment` : What the water costs
- `payment_type` : What the water costs
- `water_quality` : The quality of the water
- `quality_group` : The quality of the water
- `quantity` : The quantity of water
- `quantity_group` : The quantity of water
- `source` : The source of the water
- `source_type` : The source of the water
- `source_class` : The source of the water
- `waterpoint_type` : The kind of waterpoint
- `waterpoint_type_group` : The kind of waterpoint

### Labels

There are three possible values:

- `functional` : the waterpoint is operational and there are no repairs needed
- `functional needs repair` : the waterpoint is operational, but needs repairs
- `non functional` : the waterpoint is not operational

--- 

## Generate a submission

Your code to generate a submission file may look like this:

```python
# estimator is your model or pipeline, which you've fit on X_train

# X_test is your pandas dataframe or numpy array, 
# with the same number of rows, in the same order, as test_features.csv, 
# and the same number of columns, in the same order, as X_train

y_pred = estimator.predict(X_test)


# Makes a dataframe with two columns, id and status_group, 
# and writes to a csv file, without the index

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('your-submission-filename.csv', index=False)
```

If you're working locally, the csv file is saved in the same directory as your notebook.

If you're using Google Colab, you can use this code to download your submission csv file.

```python
from google.colab import files
files.download('your-submission-filename.csv')
```

---

In [0]:
import numpy as np
import pandas as pd

In [0]:
# checking if this is a functioning dataframe; it is
train_features;

In [0]:
# checking what datatypes we have, see if there's 
# anything categorical we need to worry about
train_features.dtypes;

In [0]:
#checking for NaN's, there's a lot, but we'll deal with them later
train_features.isna().sum();

In [0]:
# takes the features of a train and test set in a pandas dataframe. 
# Remove the target column before putting the features in here.
# returns the feature columns as a pandas dataframe one-hot-encoded

# specify which of a test or trian set you are putting in!
def one_hot_encode(train_features, test_features, max_cardinality=50):
  # necessary imports
  import pandas as pd
  import numpy as np
  import category_encoders as ce
  
  # create the one_hot_encoder
  encoder = ce.OneHotEncoder(use_cat_names = True)
  
  #determine which categorical features are appropriate to one-hot-encode
  numerics = train_features.select_dtypes(include='number').columns.tolist()
  categoricals = train_features.select_dtypes(exclude='number').columns.tolist()
  low_cardinality = [col for col in categoricals if train_features[col].nunique() <= max_cardinality]
  features = numerics + low_cardinality
  
  #create a shortened dataframe of numeric features and categorical features with low cardinality
  train_features = train_features[features]
  test_features = test_features[features]
  
  # ALWAYS .fit_transform on TRAIN data set
  # ALWAYS .transform on TEST data set
  train_features_encoded = encoder.fit_transform(train_features)
  test_features_encoded = encoder.transform(test_features)
  
  return train_features_encoded, test_features_encoded

In [0]:
# calling the one-hot-encode function on our train and test features dataframes
train_features_encoded, test_features_encoded = one_hot_encode(train_features, test_features)

In [189]:
# train, val, test split
from sklearn.model_selection import train_test_split

train_x, val_x = train_test_split(train_features_encoded, random_state = 20392)
train_y, val_y = train_test_split(train_labels, random_state = 20392)


train_x.shape, val_x.shape, train_y.shape, val_y.shape

((44550, 186), (14850, 186), (44550, 2), (14850, 2))

In [0]:
# making sure the x and y sets match up correctly
train_x.head();

In [0]:
train_y.head();

In [192]:
# getting a sense of how much accuracy our baseline model should have
train_labels['status_group'].value_counts(normalize=True)

functional                 0.543081
non functional             0.384242
functional needs repair    0.072677
Name: status_group, dtype: float64

In [0]:
# takes the column of a train_label dataframe
# prints the accuracy of the baseline
# returns the baseline list

def baseline_Logistic(labels):
  # making a list of the same size as 'labels' 
  # with only the mode as the only possible entree
  most_common_label = labels.mode()[0]
  pred_baseline = [most_common_label] * len(labels)
  
  #calculate baseline accuracy
  wrong_count = 0
  for i in range(len(pred_baseline)):
    if pred_baseline[i] != train_labels['status_group'][i]:
      wrong_count += 1
  acc = 1 - wrong_count / len(pred_baseline)
  
  print('Accuracy of Baseline: ', acc*100, '%')
  return(pred_baseline)

In [194]:
# calling the baseline function to get the baseline accuracy
pred_baseline = baseline_Logistic(train_labels['status_group'])

Accuracy of Baseline:  54.3080808080808 %


In [171]:
#on to Logistic Regression!

from sklearn.linear_model import LogisticRegression

logistic_reg = LogisticRegression(solver='lbfgs',  multi_class='multinomial', max_iter=10000)

logistic_reg.fit(train_x, train_y['status_group'])

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [195]:
print('Validation Accuracy', logistic_reg.score(val_x, val_y['status_group']))

Validation Accuracy 0.6963636363636364


In [0]:
# generate submission

# estimator is your model or pipeline, which you've fit on X_train

# X_test is your pandas dataframe or numpy array, 
# with the same number of rows, in the same order, as test_features.csv, 
# and the same number of columns, in the same order, as X_train


# y_pred is an array, we make it into a column in our submission dataframe next
y_pred = logistic_reg.predict(test_features_encoded)

# Makes a dataframe with two columns, id and status_group, 
# and writes to a csv file, without the index
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('Curtis-McKendrick-Pipes.csv', index=False)
# If you're working locally, the csv file is saved in the same directory as your notebook.


# If you're using Google Colab, you can use this code to download your submission csv file.
# from google.colab import files
# files.download('module4/Curtis-McKendrick-Pipes.csv')

In [228]:
# make sure the y_pred entrees overwrite the status_group column
# of the submission correctly
y_pred[:5]

array(['non functional', 'functional', 'non functional', 'non functional',
       'functional'], dtype=object)

In [226]:
submission.head()

,id,status_group
0,50785,non functional
1,51630,functional
2,17168,non functional
3,45559,non functional
4,49871,functional
